In [2]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
%matplotlib inline

In [97]:
df

,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,...,Music_karaoke,Music_live,Music_no_music,Music_video,OutdoorSeating,RestaurantsDelivery,RestaurantsGoodForGroups,Smoking,review_count_greater_median,cuisine_Chinese
EDqCEAGXVGCH4FJXgqtjqg,"[Restaurants, Pizza, Chicken Wings, Italian]",Toronto,"[Monday 11:0-2:0, Tuesday 11:0-2:0, Wednesday ...",1,43.661054,-79.429089,Pizza Pizza,Dufferin Grove,M6H 1L5,7,...,NaN,NaN,False,False,False,False,False,False,False,1
GDnbt3isfhd57T1QqU6flg,"[Tex-Mex, Mexican, Fast Food, Restaurants]",Scottsdale,"[Monday 10:0-22:0, Tuesday 10:0-22:0, Wednesda...",1,33.586710,-111.835410,Taco Bell,,85259,9,...,NaN,NaN,False,False,False,False,False,False,False,1
a1Ba6XeIOP48e64YFD0dMw,"[Sandwiches, Breakfast & Brunch, Salad, Restau...",Montréal,"[Monday 6:30-17:0, Tuesday 6:30-17:0, Wednesda...",1,45.502346,-73.573807,La Prep,Ville-Marie,H3A 2Z6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
826djy6K_9Fp0ptqJ2_Yag,"[Fast Food, Mexican, Restaurants]",Toronto,"[Monday 10:45-22:0, Tuesday 10:45-22:0, Wednes...",1,43.644920,-79.383333,Chipotle Mexican Grill,Downtown Core,M5J 2M2,68,...,NaN,NaN,False,True,True,False,False,True,True,1
Mi5uhdFB9OJteXPd0_IKfw,"[Restaurants, Italian, Seafood]",Glendale,"[Monday 16:0-22:0, Tuesday 16:0-22:0, Wednesda...",1,33.639301,-112.180893,Carrabba's Italian Grill,,85308,119,...,NaN,NaN,True,False,True,False,False,True,True,1
Uxh0fXFH_QQBivRnIBpdiw,"[Restaurants, Mexican, American (Traditional)]",Amherst,"[Monday 11:0-22:0, Tuesday 11:0-22:0, Wednesda...",1,41.416152,-82.207603,Don Tequila,,44001,22,...,NaN,NaN,True,False,True,False,False,True,False,1
YPavuOh2XsnRbLfl0DH2lQ,"[Restaurants, Waffles, Southern, Soul Food]",Phoenix,"[Monday 9:0-22:0, Tuesday 9:0-22:0, Wednesday ...",1,33.435372,-112.073970,Lo-Lo's Chicken & Waffles,,85003,1276,...,NaN,NaN,True,False,True,False,True,True,True,1
hovoWva_UjbnyLWEbnFvBw,"[Thai, Restaurants]",Stow,"[Monday 11:0-21:30, Tuesday 11:0-21:30, Wednes...",1,41.165709,-81.441370,Thai Gourmet,,44224,68,...,NaN,NaN,True,False,True,False,False,True,True,1
L_thK7r3K_h5M4tV7amEKQ,"[Diners, Italian, Sandwiches, Breakfast & Brun...",Toronto,"[Monday 0:0-0:0, Tuesday 0:0-0:0, Wednesday 0:...",1,43.822982,-79.247915,Honey B Hives Restaurant,Scarborough,M1V 4C3,118,...,NaN,NaN,True,False,False,False,False,True,True,1
hEcn9k6ONd5n2mq0lB2aew,"[American (Traditional), Restaurants]",Phoenix,"[Monday 6:0-22:0, Tuesday 6:0-22:0, Wednesday ...",1,33.552767,-112.135969,JB's Restaurant,,85051,24,...,NaN,NaN,False,False,True,False,False,True,False,1


In [3]:
df = pd.read_pickle('df_1518.pkl')

In [4]:
spatial_label = pd.read_pickle('../data_processeing/spatial_labels.pkl')

In [5]:
df_new = pd.concat([df,spatial_label], axis=1)

In [6]:
df_select = df_new[(df_new['stars'] >= 4) & (df_new['spatial_label'] == 3) & (df_new['cuisine_Chinese'] == 2)]

# Spectual Clustering

In [12]:
X = df_select.loc[:, 'AgesAllowed': 'review_count_greater_median']
X = pd.concat([X, df_select[['stars']]], axis=1)
X['stars'] = X['stars'].apply(str)
X = pd.get_dummies(X, dummy_na=False, drop_first=True)
def true_false(x):
    if x == True:
        return 1
    else:
        return 0
X['review_count_greater_median'] = X['review_count_greater_median'].apply(true_false)

In [32]:
for n_clusters in range(2,5):  
    spectural_clustering = SpectralClustering(n_clusters=n_clusters, random_state=100, affinity='sigmoid').fit(X)
    labels = spectural_clustering.labels_
    print silhouette_score(X, labels, metric='cityblock')
    print list(labels).count(0)
    print list(labels).count(1)
    print list(labels).count(2)
    print list(labels).count(3)
    print list(labels).count(4)

0.217059188173
223
12
0
0
0
0.118840713371
163
66
6
0
0
0.0865960643722
141
69
11
14
0


In [33]:
spectural_clustering = SpectralClustering(n_clusters=2, random_state=999, affinity='sigmoid').fit(X)
labels_spectural = spectural_clustering.labels_

In [34]:
df_select.shape

(235, 67)

# KMeans

In [35]:
## Dictionary to store anomaly results
dic_anomaly = {}

In [36]:
## function that gets silhouette scores for a clustering method
def get_silhouette_score(X, cluster_method, method='complete'):
    
    #Choose a range(list) of clusters I would like to try:
    range_n_clusters = range(2,10)
    
    if cluster_method == 'kmeans':
        for n_clusters in range_n_clusters: 
            km_result = KMeans(n_clusters=n_clusters, random_state=22).fit(X) # fit model
            cluster_labels = km_result.labels_

            # clustering model & clustering result to variable "cluster_labels". 
            silhouette_avg = silhouette_score(X, cluster_labels, random_state=22)
            print("For n_clusters ={},".format(n_clusters)+"{} - average silhouette_score :{}".\
                  format(cluster_method, silhouette_avg))

        
    elif cluster_method == 'gaussian_mix':
        for n_clusters in range_n_clusters: 
            gm = GaussianMixture(n_components=n_clusters, random_state=22).fit(X) # fit model
            cluster_labels = gm.predict(X)

            # clustering model & clustering result to variable "cluster_labels". 
            silhouette_avg = silhouette_score(X, cluster_labels, random_state=22)
            print("For n_clusters ={},".format(n_clusters)+"{} - average silhouette_score :{}".\
                  format(cluster_method, silhouette_avg))
        
        # res_mix_cluster.predict(X)
        
    elif cluster_method == 'hierarchical':
        # Define Z
        Z = linkage(X, method)

        for n_clusters in range_n_clusters:   
            cluster_labels=  fcluster(Z, n_clusters, criterion='maxclust') 

            silhouette_avg = silhouette_score(X, cluster_labels, random_state=22)
            print("For n_clusters ={},".format(n_clusters)+"{} - average silhouette_score :{}".\
                  format(cluster_method, silhouette_avg))

In [37]:
## choose the number of clusters using silhouette method
get_silhouette_score(X, 'kmeans')

For n_clusters =2,kmeans - average silhouette_score :0.14333626835
For n_clusters =3,kmeans - average silhouette_score :0.124606938754
For n_clusters =4,kmeans - average silhouette_score :0.106677619927
For n_clusters =5,kmeans - average silhouette_score :0.120790156938
For n_clusters =6,kmeans - average silhouette_score :0.106533678219
For n_clusters =7,kmeans - average silhouette_score :0.108980657389
For n_clusters =8,kmeans - average silhouette_score :0.101234949656
For n_clusters =9,kmeans - average silhouette_score :0.0756416849073


In [43]:
n_clusters = 2  # number of clusters
#XX= X.ix[:, ] # hour of day data

#train the model.
km=KMeans(n_clusters=n_clusters, random_state=22).fit(X)
labels_km = km.labels_

print list(labels_km).count(0)
print list(labels_km).count(1)

116
119


In [45]:
## Distance 
# KM=KMeans(n_clusters=k,random_state=9)
scor=km.fit_predict(X)

res_p=pd.DataFrame(km.transform(X))  ## distances of each data point to each cluster center!!!
res_p=pd.concat((res_p, pd.DataFrame(km.fit_predict(X))),axis=1)

res_p.columns=list(range(n_clusters))+["cluster"]
res_p.loc[:,"score"]=res_p.apply(lambda x: x[int(x["cluster"])],axis=1)
res_p.sort_values("score",ascending=False)[:5]

,0,1,cluster,score
223,3.220044,3.131738,1,3.131738
120,3.374304,2.942197,1,2.942197
188,2.919564,3.001577,0,2.919564
129,2.805117,3.019720,0,2.805117
209,2.808189,2.782185,1,2.782185


In [52]:
X_df = X.copy()

In [53]:
X_df['km'] = labels_km

In [54]:
X_df['distance_KM'] = res_p.score.values

In [55]:
## Get Anomalies
def get_anomaly(df, label_col, dist_or_likelihood, thres):
    
    df_anomalies = pd.DataFrame(columns=df.columns)
    
    clustermean = df.groupby(label_col)[dist_or_likelihood].mean()
    clusterstd = df.groupby(label_col)[dist_or_likelihood].std()
    
    for label in df[label_col].unique():
        anomaly = df[(df[label_col]==label) \
               & (abs(df[dist_or_likelihood] - clustermean[label]) / clusterstd[label] > thres)]
        
        df_anomalies = pd.concat([df_anomalies, anomaly], axis=0)
    
    return df_anomalies

In [59]:
## get anomalies based on kmeans distance
km_anomalies = get_anomaly(X_df, 'km', 'distance_KM', 2)

In [60]:
km_anomalies['distance_KM']

,review_count_greater_median,AgesAllowed_True,Ambience_classy_full_bar,Ambience_classy_none,Ambience_divey_True,Ambience_hipster_loud,Ambience_hipster_quiet,Ambience_hipster_very_loud,Ambience_romantic_dressy,Ambience_romantic_formal,...,Music_no_music_True,Music_video_True,OutdoorSeating_True,RestaurantsDelivery_True,RestaurantsGoodForGroups_True,Smoking_True,stars_4.5,stars_5.0,km,distance_KM
WKayXylpXcAEReoqgNK3tA,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.689006
dy0O7DaSUd49NZWSEtvz_g,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.730364
zHNk2BnANwkvHvaXD7yYsg,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.736672
bGuxRBRKv7i1BKKfGbjxEw,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.805117
GkY6UWWn0Fz2ehcuBp66pg,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.366998
D9bf5U0y_0MnL9wMFc4bKQ,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.749243
B5JCODx4728Ce2Qca1NFHw,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.919564
E0iHvHraTa-t6ka9rYL8uQ,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.942197
1pttL4MkpxOL6Mj2azOjVQ,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.728816
ZDDzKXN_JXe7694zePRuTQ,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.782185


# Gaussian Mixture

In [61]:
##### Gaussian Mixture #########

## choose the number of clusters using silhouette method
get_silhouette_score(X, 'gaussian_mix')

For n_clusters =2,gaussian_mix - average silhouette_score :0.0700811363279
For n_clusters =3,gaussian_mix - average silhouette_score :0.0463177492177
For n_clusters =4,gaussian_mix - average silhouette_score :0.0921314493081
For n_clusters =5,gaussian_mix - average silhouette_score :0.101691581721
For n_clusters =6,gaussian_mix - average silhouette_score :0.0959442789522
For n_clusters =7,gaussian_mix - average silhouette_score :0.0892368458572
For n_clusters =8,gaussian_mix - average silhouette_score :0.0671194726295
For n_clusters =9,gaussian_mix - average silhouette_score :0.0890050267596


In [64]:
k = 5
GM=GaussianMixture(n_components=k,random_state=22)
GM.fit(X)

label_gm = GM.predict(X)

In [65]:
print list(label_gm).count(0)
print list(label_gm).count(1)
print list(label_gm).count(2)
print list(label_gm).count(3)
print list(label_gm).count(4)

72
45
16
80
22


# Isolation Forest

In [78]:
### 3) Isolation Foreset
from sklearn.ensemble import IsolationForest

# fit the model
ISF = IsolationForest(max_samples=100, random_state=22)
ISF.fit(X)

## compute anomaly score of the input. The lower, the more abnormal.
score_isf = ISF.decision_function(X)

In [80]:
score_isf.argmin()

223

# KNN Distance

In [88]:
def dist2knn(x, nn, k):
    dist_ = []
    for i in range(len(nn)):
        dist_.append(distance.euclidean(x, nn.iloc[i,:]))
    dist_.sort()
    return sum(dist_[:k+1]) # +1: remove 0 self

In [92]:
dist_sum_knn = []
for i in range(len(X)):
    print '\r{}%'.format(100.0*(i+1)/len(X)),
    dist_sum_knn.append(dist2knn(X.iloc[i,:], X, 5))

100.0%


# Output

In [85]:
result = pd.DataFrame(index=X.index)

In [94]:
result['clusters_sp'] = labels_spectural
result['clusters_km'] = labels_km
result['distance_km'] = km_anomalies['distance_KM']
result['clusters_gm'] = label_gm
result['scores_isf'] = score_isf
result['distance_knn'] = dist_sum_knn

In [95]:
result

,clusters_sp,clusters_km,clusters_gm,scores_isf,distance_km,distance_knn
KeQ1cK564cL5C_hBTFrqnA,0,0,2,-0.026406,NaN,12.213128
f5xm2RiwLv0gbmXU4BkrGA,0,1,4,0.036797,NaN,10.000000
q0oPX1DXW86QytTvvrD9MA,0,0,2,0.012530,NaN,10.708204
0C6uuZBD7xKS-uucgwoyQw,0,1,1,0.026864,NaN,11.157694
mevOvXMnlTLbSl4w8fcG2A,0,0,3,0.053299,NaN,8.928203
nTQhgQZa7HhIpyM0ICC1mw,0,0,3,0.023877,NaN,10.204187
TBzgzTFSa7pJXiLD7emYaQ,0,0,3,0.015436,NaN,9.700170
0XlbdDkYzC14SZF_bUGeXA,0,1,0,0.084316,NaN,7.292529
o5t33JVLpZWf269AEPB95Q,0,0,3,0.044613,NaN,8.878315
Ih7a0qOMLFnYAec8qyq2Gg,0,1,0,0.036159,NaN,7.878315


In [87]:
result.to_csv('torronto_chi_results.csv', index_label=False, encoding='utf-8' )

,clusters_sp,clusters_km,clusters_gm,scores_isf
KeQ1cK564cL5C_hBTFrqnA,0,0,2,-0.026406
f5xm2RiwLv0gbmXU4BkrGA,0,1,4,0.036797
q0oPX1DXW86QytTvvrD9MA,0,0,2,0.012530
0C6uuZBD7xKS-uucgwoyQw,0,1,1,0.026864
mevOvXMnlTLbSl4w8fcG2A,0,0,3,0.053299
nTQhgQZa7HhIpyM0ICC1mw,0,0,3,0.023877
TBzgzTFSa7pJXiLD7emYaQ,0,0,3,0.015436
0XlbdDkYzC14SZF_bUGeXA,0,1,0,0.084316
o5t33JVLpZWf269AEPB95Q,0,0,3,0.044613
Ih7a0qOMLFnYAec8qyq2Gg,0,1,0,0.036159
